In [1]:
from __future__ import division

import os
import zipfile

In [2]:
### SET YOUR TEAM NAME HERE! Use this folder to save intermediate results
teamname = 'Machine_Contact_Enterprise_SETI'
mydatafolder = os.path.join( os.environ['PWD'], teamname )      
input_png_folder = mydatafolder + '/cleanPngsTest'
output_zip_folder = mydatafolder + '/zippedPngsTestDay99'
workingIndexFile = os.path.join( mydatafolder + '/public_list_basic_v2_26may_2017.csv' )

In [3]:
#open(workingIndexFile).read().split(',')

# Create Training / Test sets

Using the `basic` list, we'll create training and test sets for each signal class. Then we'll archive the `.png` files into a handful of `.zip` files (We need the .zip files to be smaller than 25 MB each because there is a limitation with the size of batches of data that are uploaded to Watson Visual Recognition when training a classifier (200 MB total).)

In [4]:
# Grab the Basic file list in order to 
# Organize the Data into classes if it is a training set

indexfile_rows = open(workingIndexFile).readlines()
                                                    
uuids_classes_as_list = indexfile_rows  [1:]#slice off the first line (header)

def row_to_json(row):
    uuid,sigclass = row.strip('\n').split(',')  #strip \n and split uuid, class
    return {'uuid':uuid, 'signal_classification':sigclass}

uuids_classes_as_list = map(lambda row: row_to_json(row), uuids_classes_as_list)
print "found {} files".format(len(uuids_classes_as_list))

uuids_group_by_class = {}
for item in uuids_classes_as_list:
    uuids_group_by_class.setdefault(item['signal_classification'], []).append(item)

found 4000 files


In [10]:
#At first, use just 20 percent and 10 percent. This will be useful 
#as you prototype. You'll you use these to train Watson in the next notebook
#So, if we only do the first 20 percent and 10 percent, we can move through
#the tutorial quickly at first. Then you can come back here and increase these
#percentages.
training_percentage = 0.9
test_percentage = 0.1

assert training_percentage + test_percentage <= 1.0

training_set_group_by_class = {}
test_set_group_by_class = {}
for k, v in uuids_group_by_class.iteritems():
    
    total = len(v)
    training_size = int(total * training_percentage)
    test_size = int(total * test_percentage)
    
    training_set = v[:training_size]
    test_set = v[-1*test_size:]
    
    training_set_group_by_class[k] = training_set
    test_set_group_by_class[k] = test_set
    
    print '{}: training set size: {}'.format(k, len(training_set))
    print '{}: test set size: {}'.format(k, len(test_set))

squiggle: training set size: 900
squiggle: test set size: 100
narrowband: training set size: 900
narrowband: test set size: 100
noise: training set size: 900
noise: test set size: 100
narrowbanddrd: training set size: 900
narrowbanddrd: test set size: 100


In [11]:
training_set_group_by_class['noise'][0]

{'signal_classification': 'noise',
 'uuid': '498becc2-3693-45b3-8533-50e93532706a'}

In [12]:
if os.path.exists(output_zip_folder) is False:
    os.makedirs(output_zip_folder)

In [13]:
max_zip_file_size_in_mb = 20

In [15]:
#Create the Zip files containing the training PNG files
#Note that this limits output files to be less than <max_zip_file_size_in_mb> MB because WatsonVR has a limit on the 
#size of input files that can be sent in single HTTP calls to train a custom classifier
for k, v, in training_set_group_by_class.iteritems():
    fnames = [ os.path.join(input_png_folder, vv['uuid'] + '.dat.png') for vv in v]
    count = 1
    for fn in fnames:

        archive_name = '{}/classification_{}_{}.zip'.format(output_zip_folder, count, k)

        if os.path.exists(archive_name):
            zz = zipfile.ZipFile(archive_name, mode='a')
        else:
            print 'creating new archive', archive_name
            zz = zipfile.ZipFile(archive_name, mode='w')


        zz.write(fn, fn.split('/')[-1])
        zz.close()

        #if archive_name folder exceeds <max_zip_file_size_in_mb> MB, increase count to create a new one
        if os.path.getsize(archive_name) > max_zip_file_size_in_mb * 1024 ** 2:
            count += 1
            

OSError: [Errno 2] No such file or directory: '/gpfs/fs01/user/s18f-42b3305d048694-5fca760d00b1/notebook/work/Machine_Contact_Enterprise_SETI/cleanPngsTest/99b5c77b-62ac-4927-8bcb-9086df2d60bf.dat.png'

In [ ]:
# Create the Zip files containing the test PNG files using the following naming convention:
# testset_<NUMBER>_<CLASS>.zip (step 4 will break if a different naming convention is used)
# Refer to https://www.ibm.com/watson/developercloud/visual-recognition/api/v3/#classify_an_image for ZIP size and content limitations:
# "The max number of images in a .zip file is limited to 20, and limited to 5 MB."
for k, v, in test_set_group_by_class.iteritems():

    fnames = [outputpng_folder + '/' + vv['uuid'] + '.dat.png' for vv in v]

    # archive counter
    count = 1
    # number of image files in archive counter
    image_count = 0
    for fn in fnames:

        archive_name = '{}/testset_{}_{}.zip'.format(zipfilefolder, count, k)

        if os.path.exists(archive_name):
            if os.path.getsize(archive_name) + os.path.getsize(fn) >= 4.8 * 1024 ** 2:
                # current ZIP archive size + size of this file > max size (or at least close to); create new archive
                count += 1
                image_count = 0
                archive_name = '{}/testset_{}_{}.zip'.format(zipfilefolder, count, k)
                print 'creating new archive', archive_name
                zz = zipfile.ZipFile(archive_name, mode='w')
            else:
                zz = zipfile.ZipFile(archive_name, mode='a')
        else:
            print 'creating new archive', archive_name
            zz = zipfile.ZipFile(archive_name, mode='w')

        zz.write(fn)
        zz.close()

        image_count += 1
        # the number of files > max number of files supported by API; create new archive
        if image_count > 19:
            count +=1
            image_count = 0